5. In Chapter 4, we used logistic regression to predict the probability of default using income and balance on the `Default` data set. 
We will now estimate the test error of this logistic regression model using the validation set approach. 
Do not forget to set a random seed before beginning your analysis.

In [15]:
library(ISLR2)
library(MASS)
library(ggplot2)
library(ggthemes)
library(boot)
library(skimr)
library(caret)

# Define a color cycle to use
colors <- colorblind_pal()(8)

Loading required package: lattice


Attaching package: ‘lattice’


The following object is masked from ‘package:boot’:

    melanoma




In [2]:
head(Default)

,default,student,balance,income
,<fct>,<fct>,<dbl>,<dbl>
1,No,No,729.5265,44361.625
2,No,Yes,817.1804,12106.135
3,No,No,1073.5492,31767.139
4,No,No,529.2506,35704.494
5,No,No,785.6559,38463.496
6,No,Yes,919.5885,7491.559


In [3]:
skim(Default)

── Data Summary ────────────────────────
                           Values 
Name                       Default
Number of rows             10000  
Number of columns          4      
_______________________           
Column type frequency:            
  factor                   2      
  numeric                  2      
________________________          
Group variables            None   

── Variable type: factor ───────────────────────────────────────────────────────
  skim_variable n_missing complete_rate ordered n_unique top_counts         
1 default               0             1 FALSE          2 No: 9667, Yes: 333 
2 student               0             1 FALSE          2 No: 7056, Yes: 2944

── Variable type: numeric ──────────────────────────────────────────────────────
  skim_variable n_missing complete_rate   mean     sd    p0    p25    p50    p75
1 balance               0             1   835.   484.    0    482.   824.  1166.
2 income                0             1 33517. 13337

In [4]:
str(Default)

'data.frame':	10000 obs. of  4 variables:
 $ default: Factor w/ 2 levels "No","Yes": 1 1 1 1 1 1 1 1 1 1 ...
 $ student: Factor w/ 2 levels "No","Yes": 1 2 1 1 1 2 1 2 1 1 ...
 $ balance: num  730 817 1074 529 786 ...
 $ income : num  44362 12106 31767 35704 38463 ...


(a) Fit a logistic regression model that uses `income` and `balance` to predict `default`.

In [9]:
contrasts(Default$default)

,Yes
No,0
Yes,1


In [14]:
levels(Default$default)

[1] "No"  "Yes"

In [7]:
lr_fit <- glm(default ~ income + balance, data = Default, family = "binomial")

In [8]:
summary(lr_fit)


Call:
glm(formula = default ~ income + balance, family = "binomial", 
    data = Default)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.4725  -0.1444  -0.0574  -0.0211   3.7245  

Coefficients:
              Estimate Std. Error z value Pr(>|z|)    
(Intercept) -1.154e+01  4.348e-01 -26.545  < 2e-16 ***
income       2.081e-05  4.985e-06   4.174 2.99e-05 ***
balance      5.647e-03  2.274e-04  24.836  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 2920.6  on 9999  degrees of freedom
Residual deviance: 1579.0  on 9997  degrees of freedom
AIC: 1585

Number of Fisher Scoring iterations: 8


(b) Using the validation set approach, estimate the test error of this model. 
In order to do this, you must perform the following steps:

  i. Split the sample set into a training set and a validation set.

  ii. Fit a multiple logistic regression model using only the training observations.
  
  iii. Obtain a prediction of default status for each individual in the validation set by computing the posterior probability of default for that individual, and classifying the individual to the default category if the posterior probability is greater than 0.5.
  
  iv. Compute the validation set error, which is the fraction of the observations in the validation set that are misclassified.

In [32]:
set.seed(1)
train_rows <- createDataPartition(Default$default, p = 0.8, list = F)
X_train <- Default[train_rows, ]
X_test <- Default[-train_rows, ]
dim(X_train)
dim(X_test)

[1] 8001    4

[1] 1999    4

In [33]:
lr_fit_2 <- glm(default ~ income + balance, data = X_train, family = "binomial")

In [34]:
summary(lr_fit_2)


Call:
glm(formula = default ~ income + balance, family = "binomial", 
    data = X_train)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.4776  -0.1382  -0.0546  -0.0201   3.7460  

Coefficients:
              Estimate Std. Error z value Pr(>|z|)    
(Intercept) -1.164e+01  4.914e-01 -23.688  < 2e-16 ***
income       2.002e-05  5.611e-06   3.568 0.000359 ***
balance      5.711e-03  2.561e-04  22.304  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 2340.6  on 8000  degrees of freedom
Residual deviance: 1227.5  on 7998  degrees of freedom
AIC: 1233.5

Number of Fisher Scoring iterations: 8


In [97]:
lr_train_probs <- predict(
    lr_fit_2, X_train[, c("income", "balance")], type = "response")
lr_test_probs <- predict(
    lr_fit_2, X_test[, c("income", "balance")], type = "response"
)

In [98]:
lr_train_pred <- rep("No", nrow(X_train))
lr_test_pred <- rep("No", nrow(X_test))

lr_train_pred[lr_train_probs > 0.5] <- "Yes"
lr_test_pred[lr_test_probs > 0.5] <- "Yes"

In [99]:
cat("Train accuracy: ", mean(lr_train_pred == X_train[, "default"]), "\n")
cat("Test accuracy: ", mean(lr_test_pred == X_test[, "default"]))

Train accuracy:  0.9740032 
Test accuracy:  0.9729865

In [100]:
table(lr_test_pred, X_test[, "default"])

            
lr_test_pred   No  Yes
         No  1930   51
         Yes    3   15

In [101]:
cat("Validation set error: ", (51 + 3) / (1930 + 51 + 3 + 15), "\n")
cat("Validation set error: ", 1.0 - mean(lr_test_pred == X_test[, "default"]))

Validation set error:  0.02701351 
Validation set error:  0.02701351

(c) Repeat the process in (b) three times, using three different splits of the observations into a training set and a validation set. Comment on the results obtained.

In [103]:
fn <- function(df, target = "default", p = 0.8, seed = 1, return_list = FALSE) {

    set.seed(seed)

    train_rows <- createDataPartition(df[, target], p = p, list = return_list)
    
    X_train <- df[train_rows, ]
    X_test <- df[-train_rows, ]
    
    fit <- glm(default ~ income + balance, data = X_train, family = "binomial")
    
    train_probs <- predict(
        fit,
        X_train[, c("income", "balance")],
        type = "response"
    )
    test_probs <- predict(
        fit, 
        X_test[, c("income", "balance")],
        type = "response"
    )

    train_pred <- rep("No", nrow(X_train))
    test_pred <- rep("No", nrow(X_test))
    train_pred[train_probs > 0.5] <- "Yes"
    test_pred[test_probs > 0.5] <- "Yes"

    cat("Train accuracy: ", mean(train_pred == X_train[, target]), "\n")
    cat("Test accuracy: ", mean(test_pred == X_test[, target]), "\n")

    return(table(test_pred, X_test[, target]))
}

In [105]:
for (i in 1:3) {
    cat("Validation set number: ", i, "\n")
    confusion_matrix <- fn(Default, seed = i)
    print(confusion_matrix)
    cat("\n\n")
}

Validation set number:  1 
Train accuracy:  0.9740032 
Test accuracy:  0.9729865 
         
test_pred   No  Yes
      No  1930   51
      Yes    3   15


Validation set number:  2 
Train accuracy:  0.9737533 
Test accuracy:  0.9734867 
         
test_pred   No  Yes
      No  1923   43
      Yes   10   23


Validation set number:  3 
Train accuracy:  0.9737533 
Test accuracy:  0.9734867 
         
test_pred   No  Yes
      No  1926   46
      Yes    7   20


